# This script is to download CMIP6 data using intake-esm library

Read more from link

https://intake-esm.readthedocs.io/en/stable/tutorials/loading-cmip6-data.html

How to use intake to download GCM


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import concurrent.futures
import re
from pathlib import Path
from typing import Iterable

import chunk_util
import intake
import pandas as pd
import xarray as xr
from tqdm.autonotebook import tqdm

/tmp/ipykernel_59313/2245448372.py:10: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
# Information of all CMIP6 files that one can download from intake esm data store
url = "https://raw.githubusercontent.com/NCAR/intake-esm-datastore/master/catalogs/pangeo-cmip6.json"
url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"

In [4]:
def natural_sort(l: Iterable[str]) -> list[str]:
    """
    Sort names like r1i1p1f1, r1i2p1f1 in a natural (numeric) order.
    - r1: Realization (initial condition run),
    - i1: Initialization method,
    - p1: Physical parameters,
    - f1: External forcings.

    Numeric order means that r1i1p1f1 < r2i1p1f1 < r11i1p1f1.

    :param l: list of names to be sorted
    """

    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split("([0-9]+)", key)]
    return sorted(l, key=alphanum_key)

In [5]:
def download_files(catalog_url: str, sid: str, exp: str, var: str):
    """
    Download files from the CMIP6 data store

    :param catalog_url: intake esm data store
    :param sid: source_id
    :param exp: experiment_id
    :param var: variable_id
    """

    catalog = intake.open_esm_datastore(catalog_url)
    models = catalog.search(
        experiment_id=exp,
        table_id="Amon",
        variable_id=var,
        source_id=sid,
    )
    # then one might get several files with the same conditions
    # r1: Realization (initial condition run)
    # i1: Initialization method
    # p1: Physical parameters
    # f1: External forcings

    # if no files exist then print out error
    if len(models.df) == 0:
        print("*** No data found for", var, exp, sid)
        return False

    member_ids = natural_sort(models.df.member_id.values)

    # get the first one only then seach again
    first_member_id = member_ids[0]
    first_member = catalog.search(
        experiment_id=exp,
        table_id="Amon",
        variable_id=var,
        source_id=sid,
        member_id=first_member_id,
    )

    # if no files exist then print out error
    if len(first_member.df) == 0:
        print(
            "*** This is impossible, there must be data for",
            var,
            exp,
            sid,
            member_ids[0],
        )
        return False

    odir = Path("Download") / sid / exp
    odir.mkdir(parents=True, exist_ok=True)

    def output_file_name(key):
        return odir / f"{var}_{key}_{first_member_id}.nc"

    try:
        # If all output files exist, skip
        if all((output_file_name(key)).exists() for key in first_member.keys()):
            return True

        datasets: dict[str, xr.Dataset] = first_member.to_dataset_dict(
            xarray_open_kwargs={"consolidated": True}
        )

        for key, ds in datasets.items():
            ofile = output_file_name(key)
            tmp_ofile = ofile.with_suffix(".tmp.nc")

            # Compression
            encoding = {
                var_name: {
                    "zlib": True,
                    "complevel": 1,
                    "chunksizes": chunk_util.chunk_shape_nD(
                        data.shape, chunkSize=64 * 2**10
                    ),
                }
                for var_name, data in ds.data_vars.items()
            }

            # Save to temporary file first, and then rename to output file to
            # avoid regarding corrupted file due to sudden termination as
            # complete file.
            ds.to_netcdf(
                tmp_ofile, format="NETCDF4_CLASSIC", engine="netcdf4", encoding=encoding
            )
            tmp_ofile.rename(ofile)

    except Exception as e:
        print("*** Couldn't download", var, exp, sid, e)
        return False

    return True

In [7]:
download_files(url, "EC-Earth3", "historical", "tas")


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


In [6]:
def download_data(source_ids, experiments, variables):
    status = []

    with concurrent.futures.ProcessPoolExecutor(max_workers=5) as executor:
        futures = []
        status = []
        for sid in source_ids:
            for exp in experiments:
                for var in variables:
                    future = executor.submit(download_files, url, sid, exp, var)
                    futures.append(future)
                    status.append(
                        {
                            "source_id": sid,
                            "experiment": exp,
                            "variable": var,
                        }
                    )

        for future, stat in tqdm(zip(futures, status), total=len(futures)):
            try:
                success = future.result()
            except Exception as e:
                success = False
                print("*** Error:", e)

            stat["success"] = success

    return pd.DataFrame(status)

In [8]:
source_ids = [
    "EC-Earth3",
    "MIROC6",
    "MRI-ESM2-0",
    "ACCESS-CM2",
    "IPSL-CM6A-LR",
    "MPI-ESM1-2-HR",
]
experiments = ["historical", "ssp585"]
variables = ["tas", "ta", "ua", "va", "hur", "zg", "ts"]

download_status = download_data(source_ids, experiments, variables)
print(f"Successfully download {download_status['success'].sum()} files")
print("The following files couldn't be downloaded")
print(download_status.query("~success"))